In [1]:
import socket
import sqlite3
import configparser
import time
import signal
import sys

# Load configuration from a file
config = configparser.ConfigParser()
config.read('config.ini')

# Get the PiAware configuration parameters
piaware_ip = config.get('PiAware', 'IP')
piaware_port = config.getint('PiAware', 'Port')

# Create a SQLite database and define flight_data and navigation tables
def create_database():
    with sqlite3.connect('flight_data.db') as conn:
        cursor = conn.cursor()

        # Create the flight_data and navigation_data tables using a single loop
        for table_name in ['flight_data', 'navigation_data']:
            cursor.execute(f'''
                CREATE TABLE IF NOT EXISTS {table_name} (
                    id INTEGER PRIMARY KEY,
                    message_type TEXT,
                    aircraft_icao_id TEXT,
                    date TEXT,
                    timestamp TEXT,
                    altitude REAL,
                    latitude REAL,
                    longitude REAL,
                    speed REAL,
                    heading REAL
                )
            ''')

        conn.commit()

# Create the database and tables
create_database()

# Handle Ctrl+C to gracefully exit the program
def signal_handler(sig, frame):
    print("Exiting the program.")
    sys.exit(0)

# Create a function to handle the data stream and database operations
def data_stream_and_store():
    while True:
        try:
            # Connect to the SQLite database
            with sqlite3.connect('flight_data.db') as conn:
                cursor = conn.cursor()

                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                sock.settimeout(600)

                try:
                    sock.connect((piaware_ip, piaware_port))
                    print('Connection established')

                    while True:
                        data = sock.recv(4096)

                        if not data:
                            print("No data received.")
                            print('Restarting socket')
                            sock.close()
                            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                            sock.settimeout(600)
                            sock.connect((piaware_ip, piaware_port))
                            continue

                        data_str = data.decode('utf-8')
                        fields = data_str.split(',')

                        # Check for valid message type
                        if fields[0] == 'MSG' and (fields[1] == '3' or fields[1] == '4') and len(fields) > 20:
                            message_type = fields[0] + fields[1]
                            aircraft_icao_id, date, timestamp = fields[4], fields[6], fields[7]

                            if fields[1] == '3':
                                altitude, latitude, longitude = fields[11], fields[14], fields[15]
                                cursor.execute("INSERT INTO flight_data (message_type, aircraft_icao_id, date, timestamp, altitude, latitude, longitude) VALUES (?, ?, ?, ?, ?, ?, ?)",
                                               (message_type, aircraft_icao_id, date, timestamp, altitude, latitude, longitude))
                                print(f"Record added - Message Type: {message_type}, Aircraft: {aircraft_icao_id}, Altitude: {altitude}, Latitude: {latitude}, Longitude: {longitude}")
                            else:
                                speed, heading = fields[12], fields[13]
                                cursor.execute("INSERT INTO navigation_data (message_type, aircraft_icao_id, date, timestamp, speed, heading) VALUES (?, ?, ?, ?, ?, ?)",
                                               (message_type, aircraft_icao_id, date, timestamp, speed, heading))
                                print(f"Record added - Message Type: {message_type}, Aircraft: {aircraft_icao_id}, Speed: {speed}, Heading: {heading}")

                            conn.commit()
                            time.sleep(10)

                except socket.timeout:
                    print("Socket timeout, waiting for data...")
                    time.sleep(10)
                    print('Restarting socket')
                    sock.close()
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.settimeout(600)
                    sock.connect((piaware_ip, piaware_port))
                except Exception as e:
                    print(f"Error: {e}")

        except Exception as e:
            print(f"Outer Error: {e}")


# Start the data stream and database operations
if __name__ == "__main__":
    # Register the signal handler for Ctrl+C
    signal.signal(signal.SIGINT, signal_handler)
    
    # Start the data stream and database operations
    data_stream_and_store()


Connection established
Record added - Message Type: MSG4, Aircraft: A58672, Speed: 423, Heading: 206
Exiting the program.


SystemExit: 0

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import socket
import time
import signal
import sys
import adsb_functions


#Handle Ctrl+C to gracefully exit the program
def signal_handler(sig, frame):
    print("Exiting the program.")
    sys.exit(0)

if __name__ == "__main__":
    # Register the signal handler for Ctrl+C
    signal.signal(signal.SIGINT, signal_handler)

    # Create the database and start the data stream and database operations
    create_database()
    data_stream_and_store()


NameError: name 'create_database' is not defined

In [21]:
import sqlite3
import socket
import configparser
import time

# Load configuration from a file
config = configparser.ConfigParser()
config.read('config.ini')

# Get the PiAware configuration parameters
piaware_ip = config.get('PiAware', 'IP')
piaware_port = config.getint('PiAware', 'Port')

# Create a SQLite database and define flight_data and navigation tables
def create_database():
    with sqlite3.connect('flight_data.db') as conn:
        cursor = conn.cursor()

        # Create the flight_data and navigation_data tables using a single loop
        for table_name in ['flight_data', 'navigation_data']:
            cursor.execute(f'''
                CREATE TABLE IF NOT EXISTS {table_name} (
                    id INTEGER PRIMARY KEY,
                    message_type TEXT,
                    aircraft_icao_id TEXT,
                    date TEXT,
                    timestamp TEXT,
                    altitude REAL,
                    latitude REAL,
                    longitude REAL,
                    speed REAL,
                    heading REAL
                )
            ''')

        conn.commit()

# Create a function to handle the data stream and database operations
def data_stream_and_store():
    while True:
        try:
            # Connect to the SQLite database
            with sqlite3.connect('flight_data.db') as conn:
                cursor = conn.cursor()

                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                sock.settimeout(600)

                try:
                    sock.connect((piaware_ip, piaware_port))
                    print('Connection established')

                    while True:
                        data = sock.recv(4096)

                        if not data:
                            print("No data received.")
                            print('Restarting socket')
                            sock.close()
                            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                            sock.settimeout(600)
                            sock.connect((piaware_ip, piaware_port))
                            continue

                        data_str = data.decode('utf-8')
                        fields = data_str.split(',')

                        # Check for valid message type
                        if fields[0] == 'MSG' and (fields[1] == '3' or fields[1] == '4') and len(fields) > 20:
                            message_type = fields[0] + fields[1]
                            aircraft_icao_id, date, timestamp = fields[4], fields[6], fields[7]

                            if fields[1] == '3':
                                altitude, latitude, longitude = fields[11], fields[14], fields[15]
                                cursor.execute("INSERT INTO flight_data (message_type, aircraft_icao_id, date, timestamp, altitude, latitude, longitude) VALUES (?, ?, ?, ?, ?, ?, ?)",
                                               (message_type, aircraft_icao_id, date, timestamp, altitude, latitude, longitude))
                                print(f"Record added - Message Type: {message_type}, Aircraft: {aircraft_icao_id}, Altitude: {altitude}, Latitude: {latitude}, Longitude: {longitude}")
                            else:
                                speed, heading = fields[12], fields[13]
                                cursor.execute("INSERT INTO navigation_data (message_type, aircraft_icao_id, date, timestamp, speed, heading) VALUES (?, ?, ?, ?, ?, ?)",
                                               (message_type, aircraft_icao_id, date, timestamp, speed, heading))
                                print(f"Record added - Message Type: {message_type}, Aircraft: {aircraft_icao_id}, Speed: {speed}, Heading: {heading}")

                            conn.commit()
                            time.sleep(10)

                except socket.timeout:
                    print("Socket timeout, waiting for data...")
                    time.sleep(10)
                    print('Restarting socket')
                    sock.close()
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.settimeout(600)
                    sock.connect((piaware_ip, piaware_port))
                except Exception as e:
                    print(f"Error: {e}")

        except Exception as e:
            print(f"Outer Error: {e}")


In [28]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('flight_data.db')

# Execute an SQL query to select data from the flight_data table
query = 'SELECT * FROM flight_data'
df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

In [32]:
df.shape

(3034, 8)

In [30]:
import numpy as np
df = df.replace('', np.nan)
df.dropna(inplace=True)

In [31]:
new_df = df.drop_duplicates(subset=['aircraft_icao_id','date'])
new_df = new_df.reset_index(drop=True)
new_df.shape

(564, 8)

In [34]:
new_df.head()

,id,message_type,aircraft_icao_id,date,timestamp,altitude,latitude,longitude
0,1,MSG3,A310BB,2023/10/21,00:42:56.675,33025.0,38.34210,-89.66679
1,3,MSG3,A51B58,2023/10/21,00:43:07.598,19475.0,38.42478,-89.93286
2,6,MSG3,ABBD73,2023/10/21,00:44:08.980,36000.0,38.44894,-89.69734
3,10,MSG3,AA7A61,2023/10/21,00:46:43.895,19925.0,38.50447,-90.07356
4,14,MSG3,A38237,2023/10/21,00:48:51.451,6075.0,38.67110,-90.00627


In [5]:
import os
os.listdir()

['flight_map.html',
 'Untitled1.ipynb',
 'adsb_functions.py',
 'Untitled.ipynb',
 '__pycache__',
 'config.ini',
 'final_project.ipynb',
 'flight_data.db',
 '.ipynb_checkpoints']

In [7]:
import socket
import time
import signal
import sys
import adsb_functions


#Handle Ctrl+C to gracefully exit the program
def signal_handler(sig, frame):
    print("Exiting the program.")
    sys.exit(0)

if __name__ == "__main__":
    # Register the signal handler for Ctrl+C
    signal.signal(signal.SIGINT, signal_handler)

    # Create the database and start the data stream and database operations
    create_database()
    data_stream_and_store()

NameError: name 'create_database' is not defined

In [2]:
import socket
import sqlite3
import configparser
import time
import signal
import sys

# Load configuration from a file
config = configparser.ConfigParser()
config.read('config.ini')

# Get the PiAware configuration parameters
piaware_ip = config.get('PiAware', 'IP')
piaware_port = config.getint('PiAware', 'Port')

# Create a SQLite database and define flight_data and navigation table
def create_database():
    conn = sqlite3.connect('flight_data.db')  # Replace with the desired database name
    cursor = conn.cursor()

    # Create the flight_data table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS flight_data (
            id INTEGER PRIMARY KEY,
            message_type TEXT,
            aircraft_icao_id TEXT,
            date TEXT,
            timestamp TEXT,
            altitude REAL,
            latitude REAL,
            longitude REAL
        )
    ''')

    # Create the navigation_data table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS navigation_data (
            id INTEGER PRIMARY KEY,
            message_type TEXT,
            aircraft_icao_id TEXT,
            date TEXT,
            timestamp TEXT,
            speed REAL,
            heading REAL
        )
    ''')

    conn.commit()
    conn.close()

# Create the database and flight_data table
create_database()

# Handle Ctrl+C to gracefully exit the program
def signal_handler(signal, frame):
    print("Exiting the program.")
    sys.exit(0)

# Create a function to handle the data stream and database operations
def data_stream_and_store():
    while True:
        try:
            # Connect to the SQLite database
            conn = sqlite3.connect('flight_data.db')
            cursor = conn.cursor()

            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(600)

            try:
                sock.connect((piaware_ip, piaware_port))
                print('Connection established')

                while True:
                    try:
                        data = sock.recv(4096)
                        if not data:
                            print("No data received.")
                            print('Restarting socket')
                            sock.close()  # Close the existing socket
                            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Create a new socket
                            sock.settimeout(600)  # Set timeout for the new socket
                            sock.connect((piaware_ip, piaware_port))
                            continue  # Continue receiving data

                        data_str = data.decode('utf-8')
                        fields = data_str.split(',')

                        # Handle the case where the MSG message type is not 3 or 4
                        if not fields[0] == 'MSG' or not (fields[1] == '3' or fields[1] == '4'):
                            continue

                        # Insert the data into the correct table
                        if fields[1] == '3' and len(fields) > 20:
                            cursor.execute("INSERT INTO flight_data (message_type, aircraft_icao_id, date, timestamp, altitude, latitude, longitude) VALUES (?, ?, ?, ?, ?, ?, ?)",
                                           (fields[0] + fields[1], fields[4], fields[6], fields[7], fields[11], fields[14], fields[15]))
                            print(f"Record added - Message Type: {fields[0] + fields[1]}, Aircraft: {fields[4]}, Altitude: {fields[11]}, Latitude: {fields[14]}, Longitude: {fields[15]}")
                        elif fields[1] == '4' and len(fields) > 20:
                            cursor.execute("INSERT INTO navigation_data (message_type, aircraft_icao_id, date, timestamp, speed, heading) VALUES (?, ?, ?, ?, ?, ?)",
                                           (fields[0] + fields[1], fields[4], fields[6], fields[7], fields[12], fields[13]))
                            print(f"Record added - Message Type: {fields[0] + fields[1]}, Aircraft: {fields[4]}, Speed: {fields[12]}, Heading: {fields[13]}")

                        # Commit the changes to the database
                        conn.commit()
                        time.sleep(10)
                    except socket.timeout:
                        print("Socket timeout, waiting for data...")
                        time.sleep(10)
                        print('Restarting socket')
                        sock.close()  # Close the existing socket
                        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Create a new socket
                        sock.settimeout(600)  # Set timeout for the new socket
                        sock.connect((piaware_ip, piaware_port))
                    except Exception as e:
                        print(f"Error: {e}")
            except socket.error as e:
                print(f"Socket error: {e}")
            finally:
                cursor.close()
                conn.close()
                sock.close()
        except Exception as e:
            print(f"Outer Error: {e}")

# Start the data stream and database operations
if __name__ == "__main__":
    # Register the signal handler for Ctrl+C
    signal.signal(signal.SIGINT, signal_handler)
    
    # Start the data stream and database operations
    data_stream_and_store()

Connection established
Record added - Message Type: MSG3, Aircraft: A58672, Altitude: 30000, Latitude: 38.52887, Longitude: -89.71039
Record added - Message Type: MSG4, Aircraft: A58672, Speed: 423, Heading: 206
Record added - Message Type: MSG4, Aircraft: A58672, Speed: 423, Heading: 206
Record added - Message Type: MSG3, Aircraft: A58672, Altitude: 30000, Latitude: 38.47403, Longitude: -89.74433
Record added - Message Type: MSG3, Aircraft: A58672, Altitude: 30000, Latitude: 38.45494, Longitude: -89.75615
Record added - Message Type: MSG3, Aircraft: A58672, Altitude: 30000, Latitude: 38.43674, Longitude: -89.76740
Record added - Message Type: MSG3, Aircraft: A58672, Altitude: 30000, Latitude: 38.43585, Longitude: -89.76791
Record added - Message Type: MSG4, Aircraft: A58672, Speed: 419, Heading: 206
Record added - Message Type: MSG3, Aircraft: A58672, Altitude: 30000, Latitude: 38.39633, Longitude: -89.79233
Record added - Message Type: MSG3, Aircraft: A58672, Altitude: 30000, Latitud

SystemExit: 0